# Tinder automated like
This app allows you to send Likes on tinder automatically.

facebook_token, which expires almost in an hour, can be got in the url after pusshing OK button in https://www.facebook.com/dialog/oauth?client_id=464891386855067&redirect_uri=fbconnect://success&scope=basic_info%2Cemail%2Cpublic_profile%2Cuser_about_me%2Cuser_activities%2Cuser_birthday%2Cuser_education_history%2Cuser_friends%2Cuser_interests%2Cuser_likes%2Cuser_location%2Cuser_photos%2Cuser_relationship_details&response_type=token&__mref=message

facebook_id can be found from http://findmyfbid.com

In [ ]:
import pynder
import datetime
import sqlite3
import time
import os
import json
import getpass
import sys

In [ ]:
# copied from  https://github.com/philipperemy/Deep-Learning-Tinder/blob/master/tinder_token.py

import re
import robobrowser
import requests

MOBILE_USER_AGENT = "Mozilla/5.0 (Linux; U; en-gb; KFTHWI Build/JDQ39) AppleWebKit/535.19 (KHTML, like Gecko) Silk/3.16 Safari/535.19"
FB_AUTH = "https://www.facebook.com/v2.6/dialog/oauth?redirect_uri=fb464891386855067%3A%2F%2Fauthorize%2F&display=touch&state=%7B%22challenge%22%3A%22IUUkEUqIGud332lfu%252BMJhxL4Wlc%253D%22%2C%220_auth_logger_id%22%3A%2230F06532-A1B9-4B10-BB28-B29956C71AB1%22%2C%22com.facebook.sdk_client_state%22%3Atrue%2C%223_method%22%3A%22sfvc_auth%22%7D&scope=user_birthday%2Cuser_photos%2Cuser_education_history%2Cemail%2Cuser_relationship_details%2Cuser_friends%2Cuser_work_history%2Cuser_likes&response_type=token%2Csigned_request&default_audience=friends&return_scopes=true&auth_type=rerequest&client_id=464891386855067&ret=login&sdk=ios&logger_id=30F06532-A1B9-4B10-BB28-B29956C71AB1&ext=1470840777&hash=AeZqkIcf-NEW6vBd"

def get_fb_access_token(email, password):
    s = robobrowser.RoboBrowser(user_agent=MOBILE_USER_AGENT, parser="lxml")
    s.open(FB_AUTH)
    f = s.get_form()
    f["pass"] = password
    f["email"] = email
    s.submit_form(f)
    f = s.get_form()
    try:
        s.submit_form(f, submit=f.submit_fields['__CONFIRM__'])
        access_token = re.search(r"access_token=([\w\d]+)", s.response.content.decode()).groups()[0]
        return access_token
    except Exception as ex:
        print("access token could not be retrieved. Check your username and password.")
        print("Official error: %s" % ex)
        return {"error": "access token could not be retrieved. Check your username and password."}

def get_fb_id(access_token):
    if "error" in access_token:
        return {"error": "access token could not be retrieved"}
    """Gets facebook ID from access token"""
    req = requests.get('https://graph.facebook.com/me?access_token=' + access_token)
    return req.json()["id"]

In [ ]:
facebook_auth_token = get_fb_access_token(input('Email: '), getpass.getpass('Password: '))
facebook_id = get_fb_id(facebook_auth_token)
if 'error' in facebook_auth_token or 'error' in facebook_id:
    sys.exit(0)

In [ ]:
session = pynder.Session(facebook_id, facebook_auth_token)

In [ ]:
latlon = {'shibuya':(35.6618, 139.7041),
         'todai': (35.7127, 139.7620),
         'ginza': (35.6721, 139.7708)}

In [ ]:
#session.update_location(*latlon['shibuya']) # updates latitude and longitude for your profile

In [ ]:
#profile = session.profile  # your profile. If you update its attributes they will be updated on Tinder.

Then, add people into database. Sometimes a boy/girl is shown which you already liked. When you see a boy/girl again, skip photos, schools, jobs, common_connections and instagram_username if the same thing is already in database, otherwise insert new row.

In [ ]:
conn = sqlite3.connect('tinder.db')
conn.isolation_level = None
c = conn.cursor()

In [ ]:
c.execute('''CREATE TABLE IF NOT EXISTS girls
             (id text primary key, name text, bio text, age integer, birth_date timestamp,
             ping_time text, distance_km real, like integer, match integer)''')
c.execute('''CREATE TABLE IF NOT EXISTS photos
             (id text, photo_url text, scraped_date date, unique(id, photo_url))''')
c.execute('''CREATE TABLE IF NOT EXISTS schools
             (id text, school text, scraped_date date, unique(id, school))''')
c.execute('''CREATE TABLE IF NOT EXISTS jobs
             (id text, job text, scraped_date date, unique(id, job))''')
c.execute('''CREATE TABLE IF NOT EXISTS common_connections
             (id text, friend text, scraped_date date, unique(id, friend))''')
c.execute('''CREATE TABLE IF NOT EXISTS instagram_username
             (id text, username text, scraped_date date, unique(id, username))''')

In [ ]:
for user in users:
    if (any([school for school in user.schools]))
    or ('common_friends' in user._data and len(user.common_connections)>8):
        print(1)
    else:
        print(0)

In [ ]:
cnt = 0
today = datetime.datetime.today()
while session.can_like_in < 0:
    try:
        users = session.nearby_users() # returns a list of users nearby
    except KeyError as e:
        print('KeyError: ', e)
        time.sleep(10)
        continue
    # like
    for user in users:
        if any(['university of tokyo' in school.lower() for school in user.schools]) or\
        any(['utokyo' in school.lower() for school in user.schools]) or\
        ('common_friends' in user._data and len(user.common_connections)>8):
            try:
                user.superlike()
            except:
                user.like()
        else:
            user.like()
    # put database
    girls = [(user.id, user.name, user.bio, user.age, user.birth_date, datetime.datetime.strptime(user.ping_time, '%Y-%m-%dT%H:%M:%S.%fZ'),
             user.distance_km, 1, 0) for user in users]
    photos = [(user.id, photo, today) for user in users for photo in user.get_photos(width='640') if user.photos] # (id, photo_url)
    schools = [(user.id, school, today) for user in users for school in user.schools] # (id, school)
    jobs = [(user.id, job, today) for user in users for job in user.jobs] # (id, job)
    common_connections = [(user.id, conn, today) for user in users if 'common_friends' in user._data for conn in user.common_connections] # (id, friend)
    instagram_username = [(user.id, user.instagram_username, today) for user in users if user.instagram_username] # (id, username)
    try:
        c.execute('begin')
        c.executemany('INSERT OR REPLACE INTO girls VALUES (?,?,?,?,?,?,?,?,?)', girls)
        c.executemany('INSERT OR IGNORE INTO photos VALUES (?,?,?)', photos)
        c.executemany('INSERT OR IGNORE INTO schools VALUES (?,?,?)', schools)    
        c.executemany('INSERT OR IGNORE INTO jobs VALUES (?,?,?)', jobs)
        c.executemany('INSERT OR IGNORE INTO common_connections VALUES (?,?,?)', common_connections)
        c.executemany('INSERT OR IGNORE INTO instagram_username VALUES (?,?,?)', instagram_username)
        c.execute('end')
    except:
        c.execute('rollback')
    cnt += len(users)
    time.sleep(0.5)
    print('You liked {} girls.'.format(cnt))
nexttime = datetime.datetime.now() + datetime.timedelta(seconds=session.can_like_in)
print('You can like again at '+nexttime.strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
conn.commit()
conn.close()